# Scrape AI Wiki

Author: Crystal

Date: June 21, 2021

In [89]:
import pandas as pd
import numpy as np
import json
import glob
import time


# web scrapping
import requests as r
from bs4 import BeautifulSoup
import re
import bs4 as bs
import urllib

import nltk


/home/zz3hs/.conda/envs/crystal/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [78]:
url = 'https://en.wikipedia.org/wiki/Artificial_intelligence'
source = urllib.request.urlopen(url).read()

#parse the data and create beautifulsoup object
soup = BeautifulSoup(source, 'html.parser')

# Scrapping all the text

In [83]:
# all the text
text = ""
for paragraph in soup.find_all('p'):
    text += paragraph.text

#print(text)

text = re.sub(r'\[[0-9]*\]', ' ', text) #removes numbers
text = re.sub(r'\s+',' ',text) #remove long blank space
text = text.lower() #lowercase text
text = re.sub(r'\d', ' ', text) 
text = re.sub(r'\s+', ' ', text)
#print(text)

sentences = nltk.sent_tokenize(text)
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/zz3hs/nltk_data'
    - '/home/zz3hs/.conda/envs/crystal/nltk_data'
    - '/home/zz3hs/.conda/envs/crystal/share/nltk_data'
    - '/home/zz3hs/.conda/envs/crystal/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


# Scrapping table of contents

In [4]:
# nested items listed in the table of contents
topics = []
for topic in soup.find_all("h3"):
    topics.append(topic.text)
print(topics)

['Reasoning, problem solving', 'Knowledge representation', 'Planning', 'Learning', 'Natural language processing', 'Perception', 'Motion and manipulation', 'Social intelligence', 'General intelligence', 'Cybernetics and brain simulation', 'Symbolic', 'Sub-symbolic', 'Statistical', 'Integrating the approaches', 'The limits of artificial general intelligence', 'Ethical machines', 'Machine consciousness, sentience and mind', 'Superintelligence', 'Risks of narrow AI', 'Risks of general AI', 'AI textbooks', 'History of AI', 'Other sources', '\nPersonal tools\n', '\nNamespaces\n', '\nVariants\n', '\nViews\n', '\nMore\n', '\nSearch\n', '\nNavigation\n', '\nContribute\n', '\nTools\n', '\nPrint/export\n', '\nIn other projects\n', '\nLanguages\n']


# Scrapping all the links

1. Exclude url links that contain:
    - ".png"
    - ".jpg"
    - ".svg"
    - ".gif"
    - "Template"
    - "(identifier)", might be a link to a journal article


2. Exclude duplicate links

In [73]:
url_ls = []
title_ls = []

condition = url.startswith("/wiki/") & (not url.endswith(".jpg"))  & (not url.endswith(".png"))  & (not "Template" in url) & (not url.endswith(".gif"))  & (not "(identifier)" in url)

for link in soup.find_all("a"):
    url = link.get("href", "")
    if url.startswith("/wiki/") & (not url.endswith(".jpg"))  & (not url.endswith(".png"))  & (not "Template" in url) & (not url.endswith(".gif"))  & (not "(identifier)" in url):
       # print(url)
        url_ls.append(url)
        title_ls.append(link.text.strip())


df = pd.DataFrame({'url':url_ls, 'title':title_ls})
df = df.drop_duplicates(subset = 'url', keep = 'first')

print(df.shape)
df.to_csv(r'/home/zz3hs/git/dspg21RnD/data/dspg21RnD/wiki_ai_links.csv', index = False)   

(1584, 2)


Todo: 
1. Remove unrelated items
    - people's names
2. What to do with the topics listed under "Articles related to Artificial intelligence"?
3. What to do with acronames such as "k-NN"?

# Scrapping topics within the itemized list

Problem: suppose to grab three sections with class sidebar-content hlist, but only grabbing the first one


## I. AI, ML/data mining

In [28]:
# getting the headding too
div = soup.find('div', {"class": "sidebar-list-content mw-collapsible-content"})
topics = []
for topic in div.find_all("a"):
    topics.append(topic.text)
print(topics)

['Artificial general intelligence', 'Planning', 'Computer vision', 'General game playing', 'Knowledge reasoning', 'Machine learning', 'Natural language processing', 'Robotics']


In [26]:
# Problem: suppose to grab all sections with class sidebar-content, but only grabbing the first one
# getting the items only (not heading)
td = soup.find('td', {"class": "sidebar-content"})
topics = []
for topic in td.find_all("a"):
    topics.append(topic.text)
print(topics)

['Major goals', 'Artificial general intelligence', 'Planning', 'Computer vision', 'General game playing', 'Knowledge reasoning', 'Machine learning', 'Natural language processing', 'Robotics']


## II. Evolutionary algorithm

In [27]:
# part of a series on evolutionary algorithm
td = soup.find('td', {"class": "sidebar-content hlist"})
topics = []
for topic in td.find_all("a"):
    topics.append(topic.text)
print(topics)

['Artificial development', 'Artificial life', 'Cellular evolutionary algorithm', 'Cultural algorithm', 'Differential evolution', 'Effective fitness', 'Evolutionary computation', 'Evolution strategy', 'Gaussian adaptation', 'Evolutionary multimodal optimization', 'Particle swarm optimization', 'Memetic algorithm', 'Natural evolution strategy', 'Neuroevolution', 'Promoter based genetic algorithm', 'Spiral optimization algorithm', 'Self-modifying code', 'Polymorphic code']
